In [1]:
#installing required libraries
install.packages("RSQLite")
install.packages("DBI")
library(RSQLite)
library(DBI)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
#Establishing database connection
con <- dbConnect(RSQLite::SQLite(), "FinalDB.sqlite")
con

<SQLiteConnection>
  Path: /Users/shishir__/Desktop/Data Science/CourseProject/CropDataAnalysis/FinalDB.sqlite
  Extensions: TRUE

In [29]:
#creating tables
df1 <- dbExecute(con, 
                 "CREATE TABLE CROP_DATA (
                   CD_ID INTEGER NOT NULL,
                   YEAR DATE NOT NULL,
                   CROP_TYPE VARCHAR(20) NOT NULL,
                   GEO VARCHAR(20) NOT NULL, 
                   SEEDED_AREA INTEGER NOT NULL,
                   HARVESTED_AREA INTEGER NOT NULL,
                   PRODUCTION INTEGER NOT NULL,
                   AVG_YIELD INTEGER NOT NULL,
                   PRIMARY KEY (CD_ID)
                 )"
)

df2 <- dbExecute(con,
                 "CREATE TABLE FARM_PRICES(
                   CD_ID INTEGER NOT NULL,
                   DATE DATE NOT NULL,
                   CROP_TYPE VARCHAR(50) NOT NULL,
                   GEO VARCHAR(20) NOT NULL,
                   PRICE_PRERMT FLOAT NOT NULL,
                   PRIMARY KEY (CD_ID)
                 )",
                 errors = FALSE
)

df3 <- dbExecute(con, 
                 "CREATE TABLE DAILY_FX (
                   DFX_ID INTEGER NOT NULL,
                   DATE DATE NOT NULL,
                   FXUSDCAD FLOAT NOT NULL,
                   PRIMARY KEY (DFX_ID)
                 )"
)

df4 <- dbExecute(con, 
                 "CREATE TABLE MONTHLY_FX (
                   DFX_ID INTEGER NOT NULL,
                   DATE DATE NOT NULL,
                   FXUSDCAD FLOAT NOT NULL,
                   PRIMARY KEY (DFX_ID)
                 )"
)

In [33]:
#Listing the created tables 
dbListTables(con)

[1] "CROP_DATA"   "DAILY_FX"    "FARM_PRICES" "MONTHLY_FX"

In [35]:
#Read the datasets into R dataframes from the url provided and loading the tables in database
#Reading data
crop_data <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv')
farm_prices <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv')
daily_fx <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv')
monthly_fx <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv')


In [37]:
#Loading into table
dbWriteTable(con,"CROP_DATA", crop_data, overwrite=TRUE, header=TRUE)
dbWriteTable(con,"FARM_PRICES", farm_prices, overwrite=TRUE, header=TRUE)
dbWriteTable(con,"DAILY_FX", daily_fx, overwrite=TRUE, header=TRUE)
dbWriteTable(con,"MONTHLY_FX", monthly_fx, overwrite=TRUE, header=TRUE)

In [39]:
#Task: How many records are in the farm prices dataset?
dbGetQuery(con, 'SELECT COUNT(CD_ID) FROM FARM_PRICES')

COUNT(CD_ID)
<int>
2678


In [41]:
#Task: Which geographies are included in the farm prices dataset?
dbGetQuery(con, 'SELECT DISTINCT(GEO) FROM FARM_PRICES')

GEO
<chr>
Alberta
Saskatchewan


In [43]:
#Task: How many hectares of Rye were harvested in Canada in 1968?
dbGetQuery(con, 'SELECT SUM(HARVESTED_AREA) FROM CROP_DATA
                 WHERE CROP_TYPE="Rye" AND YEAR="1968-12-31" AND GEO="Canada"')

SUM(HARVESTED_AREA)
<int>
274100


In [45]:
#Task: Query and display the first 6 rows of the farm prices table for Rye.
dbGetQuery(con, 'SELECT * FROM FARM_PRICES
                 WHERE CROP_TYPE="Rye"
                 LIMIT 6')

CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
<int>,<chr>,<chr>,<chr>,<dbl>
4,1985-01-01,Rye,Alberta,100.77
5,1985-01-01,Rye,Saskatchewan,109.75
10,1985-02-01,Rye,Alberta,95.05
11,1985-02-01,Rye,Saskatchewan,103.46
16,1985-03-01,Rye,Alberta,96.77
17,1985-03-01,Rye,Saskatchewan,106.38


In [47]:
#Task: Which provinces grew Barley?
dbGetQuery(con, 'SELECT DISTINCT(GEO) FROM CROP_DATA
                 WHERE CROP_TYPE="Barley"')

GEO
<chr>
Alberta
Canada
Saskatchewan


In [49]:
#Task: Find the first and last dates for the farm prices data
dbGetQuery(con, 'SELECT MIN(DATE) AS FIRST_DATE, MAX(DATE) AS LAST_DATE 
                 FROM FARM_PRICES')

FIRST_DATE,LAST_DATE
<chr>,<chr>
1985-01-01,2020-12-01


In [51]:
#Task: Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?
dbGetQuery(con, 'SELECT CROP_TYPE, PRICE_PRERMT FROM FARM_PRICES
                WHERE PRICE_PRERMT >= 350')


CROP_TYPE,PRICE_PRERMT
<chr>,<dbl>
Canola,350.04
Canola,363.84
Canola,364.36
Canola,354.58
Canola,359.99
Canola,350.47
Canola,354.05
Canola,354.82
Canola,357.37


In [53]:
#Task: Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield.
#Which crop performed best?

dbGetQuery(con,'SELECT CROP_TYPE, AVG_YIELD 
                FROM CROP_DATA
                WHERE GEO="Saskatchewan" AND YEAR="2000-12-31"
                ORDER BY AVG_YIELD DESC ')

CROP_TYPE,AVG_YIELD
<chr>,<int>
Barley,2800
Wheat,2200
Rye,2100
Canola,1400


In [55]:
#Task : Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. 
#Which crop and province had the highest average yield since the year 2000?

dbGetQuery(con, 'SELECT CROP_TYPE, GEO, AVG_YIELD FROM CROP_DATA
                WHERE YEAR >="2000"
                GROUP BY CROP_TYPE, GEO
                ORDER BY AVG_YIELD DESC')

CROP_TYPE,GEO,AVG_YIELD
<chr>,<chr>,<int>
Barley,Alberta,3000
Barley,Canada,3000
Barley,Saskatchewan,2800
Wheat,Alberta,2500
Wheat,Canada,2400
Rye,Canada,2300
Wheat,Saskatchewan,2200
Rye,Alberta,2100
Rye,Saskatchewan,2100


In [57]:
#Task: Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data
dbGetQuery(con, 'SELECT YEAR, CROP_TYPE, GEO, HARVESTED_AREA FROM CROP_DATA
                 WHERE CROP_TYPE="Wheat" AND GEO="Canada" AND
                 YEAR= (SELECT MAX(YEAR) FROM CROP_DATA)')

YEAR,CROP_TYPE,GEO,HARVESTED_AREA
<chr>,<chr>,<chr>,<int>
2020-12-31,Wheat,Canada,10017800


In [59]:
#Task: Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown
#in Saskatchewan in both Canadian and US dollars.
#Display the most recent 6 months of the data.
dbGetQuery(con,'SELECT A.DATE, (A.PRICE_PRERMT) AS CAD, (A.PRICE_PRERMT/B.FXUSDCAD) AS USD
                FROM FARM_PRICES AS A
                INNER JOIN MONTHLY_FX AS B USING(DATE)
                WHERE A.GEO= "Saskatchewan" AND A.CROP_TYPE="Canola"
                ORDER BY A.DATE DESC
                LIMIT 6')

DATE,CAD,USD
<chr>,<dbl>,<dbl>
2020-12-01,507.33,396.1128
2020-11-01,495.64,379.2718
2020-10-01,474.80,359.2965
2020-09-01,463.52,350.4057
2020-08-01,464.60,351.3827
2020-07-01,462.88,342.9122
